## DGL-LifeSci with PyTorchバックエンドを使用した結合親和性予測用Amazon SageMakerモデルのトレーニング
**Amazon SageMaker Python SDK**を使用すると、DGL-LifeSciモデルを簡単にトレーニングすることができます。この例では、[PDBBind](http://www.pdbbind.org.cn/)データセットを使ってAtomic Convolutional Networks (ACNN) [1] またはPotentialNet [2] モデルを学習させます。これらの詳細については、[DGL-Lifesciのサンプルページ](https://github.com/yoheigon/dgl-lifesci/tree/master/examples/binding_affinity_prediction)を参照してください。

[1] Gomes et al. (2017) Atomic Convolutional Networks for Predicting Protein-Ligand Binding Affinity. *arXiv preprint arXiv:1703.10603*.

[2] Feinberg et al. (2018) PotentialNet for molecular property prediction. *ACS central science* 4.11: 1520-1530.

### セットアップ
後々必要になるいくつかの変数をここで定義しておきます。

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here.
bucket = sess.default_bucket()

# IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
# You can use the Amazon SageMaker Python SDK to get the role from the notebook environment.
role = get_execution_role()

In [3]:
bucket

'sagemaker-ap-northeast-1-233488627969'

### 学習スクリプト
`main.py`は、Amazon SageMaker モデルのトレーニングに必要なすべてのコードを提供します。

In [4]:
!cat ./code/main.py

# -*- coding: utf-8 -*-
#
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

import multiprocessing
import os

import numpy as np
import json
import glob
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import dgl
from dgllife.utils.eval import Meter
from dgl.data.utils import Subset, extract_archive
from dgl.data.utils import load_graphs

from sklearn.model_selection import train_test_split
from utils import load_dataset, load_model, rand_hyperparams, set_random_seed

import pandas as pd
import boto3


def update_msg_from_scores(msg, scores):
    for metric, score in scores.items():
        msg += ", {} {:.4f}".format(metric, score)
    return msg


def run_a_train_epoch(args, epoch, model, data_loader, loss_criterion, optimizer):
    model.train()
    train_meter = Meter(args["train_mean"], args["train_std"])
    epoch_loss = 0
    for batch_id, batch_data in enumerate(data_loader):
        b

In [5]:
!aws s3 ls s3://{bucket}/preprocessed/graph_files_v2020_refined_core/

2022-12-16 03:16:07     118822 1a30_g1.bin
2022-12-16 03:16:07     353707 1a30_g2.bin
2022-12-16 03:16:07      94262 1bcu_g1.bin
2022-12-16 03:16:07     278347 1bcu_g2.bin
2022-12-16 03:16:07     118622 1bzc_g1.bin
2022-12-16 03:16:07     353467 1bzc_g2.bin
2022-12-16 03:16:07     142814 1e66_g1.bin
2022-12-16 03:16:07     428827 1e66_g2.bin
2022-12-16 03:16:07     168070 1eby_g1.bin
2022-12-16 03:16:07     506227 1eby_g2.bin
2022-12-16 03:16:07     161326 1g2k_g1.bin
2022-12-16 03:16:07     485827 1g2k_g2.bin
2022-12-16 03:16:07     127374 1gpk_g1.bin
2022-12-16 03:16:07     381307 1gpk_g2.bin
2022-12-16 03:16:07     135254 1gpn_g1.bin
2022-12-16 03:16:07     406027 1gpn_g2.bin
2022-12-16 03:16:07     184182 1h22_g1.bin
2022-12-16 03:16:07     556747 1h22_g2.bin
2022-12-16 03:16:07     191390 1h23_g1.bin
2022-12-16 03:16:07     578587 1h23_g2.bin
2022-12-16 03:16:08     115462 1k1i_g1.bin
2022-12-16 03:16:07     342907 1k1i_g2.bin
2022-12-16 03:16:07     147758 1lpg_g1.bin
2022-12-16 

### 入力データセットの指定

`1_make_potential_graphs`のノートブックでバイナリファイルを配置したのと同じS3 URIを使用します。

In [6]:
#s3_input = sagemaker.inputs.TrainingInput(s3_data='<your S3 prefix of input binary files>')
s3_input = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/preprocessed/graph_files_v2020_refined_core/')
#s3_input_ft = sagemaker.inputs.TrainingInput(s3_data='<your S3 prefix of input binary files for fine tuning>') #option

### SageMaker Estimator Class
Amazon SageMaker Estimatorを使用すると、CPUまたはGPUベースのインスタンスを使用して、Amazon SageMakerでシングルマシンを実行することができます。

Estimatorを作成する際に、学習スクリプトのファイル名とIAM実行ロールの名前を渡します。`instance_count`と`instance_type`は、トレーニングジョブに使用されるAmazon SageMakerインスタンスの数と種類を決定します。`hyperparameters`パラメータは、`argparse`を使用して解析できるように、パラメータとしてトレーニングスクリプトに渡される値のディクショナリです。これらの値にアクセスする方法は、上記の`main.py`スクリプトで確認することができます。

この例では、学習用インスタンスにml.p3.2xlargeを、PDBBind(v2020) core + generalデータセットを使用しています。

In [7]:
from sagemaker.pytorch import PyTorch

metric_definitions = [
    {"Name": "val_mae", "Regex": "val mae: ([0-9\\.]+)"},
    {"Name": "val_r2", "Regex": "val r2: ([0-9\\.]+)"},
    {"Name": "test_mae", "Regex": "test mae: ([0-9\\.]+)"},
    {"Name": "test_r2", "Regex": "test r2: ([0-9\\.]+)"},
    {"Name": "mae", "Regex": "mae ([0-9.]+).*$"},
    {"Name": "r2", "Regex": "r2 ([0-9.]+).*$"},
]

# Create estimator
estimator = PyTorch(
    entry_point="main.py",
    source_dir="code",
    role=role,
    framework_version="1.6.0",
    py_version="py3",
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    hyperparameters={
        "lr":0.001,
    },
    metric_definitions=metric_definitions,
)

### Running the Training Job
After you construct the Estimator object, fit it by using Amazon SageMaker. The [PDBBind](http://www.pdbbind.org.cn/) dataset is automatically downloaded.

In [8]:
# Launch SageMaker training job
estimator.fit({'train': s3_input})

2022-12-16 03:24:01 Starting - Starting the training job...
2022-12-16 03:24:28 Starting - Preparing the instances for trainingProfilerReport-1671161041: InProgress
............
2022-12-16 03:26:20 Downloading - Downloading input data......
2022-12-16 03:27:21 Training - Downloading the training image......
2022-12-16 03:28:21 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-16 03:28:27,768 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-16 03:28:27,806 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-16 03:28:27,810 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-12-16 03:28:27,988 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.

In [9]:
!aws s3 cp {estimator.output_path}{estimator.latest_training_job.job_name}/output/model.tar.gz .

download: s3://sagemaker-ap-northeast-1-233488627969/pytorch-training-2022-12-16-03-24-01-215/output/model.tar.gz to ./model.tar.gz


## Go to the next notebook : 3 local inference potentialnet

### (Optional) ファインチューニングジョブを実行

In [ ]:
# Create estimator
estimator_ft = PyTorch(
    entry_point="main.py",
    source_dir="code",
    role=role,
    framework_version="1.6.0",
    py_version="py3",
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    hyperparameters={
        "lr":0.001,
        "fine_tune":True,
        "pretrained_model":'<your S3 prefix of a trained model>/model.tar.gz'
    },
    metric_definitions=metric_definitions,
)

In [ ]:
# Launch SageMaker training job
estimator_ft.fit({'train': s3_input_ft})

### (Optional) ハイパーパラメータチューニングジョブを実行

SageMakerは、ハイパーパラメータの組み合わせを変えて複数の学習ジョブを開始し、最も優れたモデル性能を持つセットを見つけるHyperparameter Tuningを提供します。チューニングするハイパーパラメータとその値を指定することができます。

In [ ]:
import boto3
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.01),
    "num_epochs": IntegerParameter(100, 200),
}

次に、チューニングしたい目的メトリクスとその定義を指定します。これには、SageMaker Training JobのAmazon CloudWatch Logsからそのメトリックを抽出するために必要な正規表現（regex）が含まれます。この例では、"mae "と "r2 "がサポートされています。

In [ ]:
objective_metric_name = "mae"


ここで、HyperparameterTunerオブジェクトを作成し、以下の変数を渡します。

 * 上記で作成した学習推定量
 * ハイパーパラメータの範囲
 * 目的メトリクスの名前と定義
 * 合計で実行するトレーニングジョブの数と、同時に実行するトレーニングジョブの数。並列ジョブを増やすとチューニングが早く終わりますが、精度が犠牲になる場合があります。並列ジョブの値をトレーニングジョブの総数の10%以下に設定することをお勧めします(この例では短くするために高く設定します)。
 * 目的指標を最大化するか、最小化するか。デフォルトでは「最大化」になっているため、ここでは指定していません。
 

In [ ]:
task_tags = [{"Key": "ML Task", "Value": "DGL-Lifesci"}]
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    tags=task_tags,
    max_jobs=2,
    max_parallel_jobs=1,
)

最後に、`.fit().`メソッドを呼び出してチューニングジョブを実行しましょう。

In [ ]:
tuner.fit(wait=False)

Hyperparameter Tuningのジョブが正常に開始され、InProgressになっていることを確認するために、ジョブの状態を簡単にチェックしてみましょう。

In [ ]:
sagemaker_client = boto3.client("sagemaker")

sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)["HyperParameterTuningJobStatus"]

### Output
ハイパーパラメータのチューニングが終了すると、以下のように最適なモデルが得られます。SageMakerのハイパーパラメータチューニングジョブにより、計算量の多い学習ジョブを簡単に管理し、最適なモデル構成を見つけることができます。

In [ ]:
s3_client = boto3.client("s3")

tuner_status = sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)["HyperParameterTuningJobStatus"]

if tuner_status == "Completed":
    best_training_job_summary = sagemaker_boto_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)["BestTrainingJob"]
    best_training_job_details = sagemaker_boto_client.describe_training_job(
        TrainingJobName=best_training_job_summary["TrainingJobName"])

    trained_model_s3_uri = best_training_job_details["ModelArtifacts"]["S3ModelArtifacts"]
    s3_client.download_file(trained_model_s3_uri.split('/')[2], ('/').join(trained_model_s3_uri.split('/')[3:]), "model.tar.gz")